# Creating a grid of photometric fluxes

This tutorial shows how to generate a grid of photometric fluxes 
from a given SSP model, consisting of four dimensions:
- Observed redshift
- Dust attenuation
- Filter band
- SSP metallicity
- SSP age

The SSP photometric grid can then be combined with a given
star formation history, e.g. a log-normal SFH, to produce a grid of photometric fluxes
for different SFHs.


In [1]:
from matplotlib import pyplot as plt
import os
import numpy as np
from time import time
from astropy import units as u

# Import the SSP model
from pst.SSP import PopStar
# Import the dust extinction model
from pst.dust import DustScreen
# Import the observables
from pst.observables import Filter, load_photometric_filters
# Import the chemical evolution model
from pst.models import LogNormalCEM

First, prepare the list of filters that are going to be produced. In this example, we'll use the PANSTARRS broad band filters g, r, i and z. 

In [2]:
filters = load_photometric_filters(['PANSTARRS_PS1.g', 'PANSTARRS_PS1.r', 'PANSTARRS_PS1.i', 'PANSTARRS_PS1.z'])

Next, we will initialise the SSP model. We are going to use the PopStar SSP model and the Chabrier 2003 IFM.

In [3]:
ssp = PopStar(IMF='cha')

> Initialising Popstar models (IMF='cha')
--> Only stellar continuum


To speed the computation up, we are going to decrease the spectra resolution (to 5 AA) and limit the wavelength range (1000-20000 AA).

In [4]:
ssp.interpolate_sed(np.arange(1000, 2e4, 5.0) * u.angstrom)

[SSP] Interpolating SSP SEDs


We are also interested on modelling the effects of dust attenuation on the broad band photometry. Let's create a dust screen model based on Cardelli+89 extinction law

In [5]:
dust_model = DustScreen("ccm89")

Finally, we may also want to include the effects of redshift on the observed photometry. We can declare the array of extinctions (in terms of $A_V$) and the range of observed redshifts that are going to use to generate the grid.

In [6]:
a_v_array = np.linspace(0, 2, 20)
redshift_array = np.linspace(0, 2.5, 30)

Now it is time to produce the grid

In [7]:
tstart = time()
all_photometry = np.zeros(
    (redshift_array.size, a_v_array.size, len(filters),
                           *ssp.L_lambda.shape[:-1]))

for i, z in enumerate(redshift_array):
    for j, av in enumerate(a_v_array):
        # For each value of AV, we create a new SSP model
        red_ssp = dust_model.redden_ssp_model(ssp, a_v=av)
        # Compute the SSP photometry at the observed redhisft z
        all_photometry[i, j] = red_ssp.compute_photometry(filter_list=filters,
                                                          z_obs=z)

tend = time()
print("time spent generating SSP photometry: ", tend - tstart)
print("time spent generating a single SSP model photometry: ",
      (tend - tstart) / (all_photometry.shape[0] * all_photometry.shape[1]))

Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing 

Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing 

Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing 

Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing synthetic photometry for SSP model
Computing 

Then we end up with a grid that has four dimensions (redshift, Av, filter, metallicity, age)

In [8]:
all_photometry.shape

(30, 20, 4, 6, 106)

## Composite Stellar Populations photometry

Now we can increase the complexity and combine the SSP photometric fluxes by using some chemical evolution model. 

In this example, we are going to use a log-normal star formation history with a metallicity evolution model proportional to the stellar mass ($Z(t)=Z_{today} \left(M(t) / M_{today}\right)^\alpha$

In [9]:
model = LogNormalCEM(alpha_powerlaw=0, ism_metallicity_today=0.02, lnt0=3,
                     scale=1, mass_today=1e9 * u.Msun)

Since we are interested on coverint a wide range of possible SFHs, we will generate another grid in terms of the free parameters of the SFH

In [10]:
lnt_array = np.linspace(np.log(0.1), np.log(10), 10)
scale_array = np.linspace(0.1, 5, 20)

all_models_photometry = np.zeros(
    (lnt_array.size, scale_array.size, *all_photometry.shape[:-2]))

In [13]:
tstart = time()
for i, lnt in enumerate(lnt_array):
    for j, scale in enumerate(scale_array):
        model.lnt0 = lnt
        model.scale = scale

        all_models_photometry[i, j] = model.compute_photometry(
            ssp, t_obs=ssp.ages.max(), photometry=all_photometry)

tend = time()
print("time spent generating Model photometry: ", tend - tstart)


Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming input photometry array in Jy/Msun
Assuming in